In [1]:
# https://developers.google.com/earth-engine/tutorials/community/imad-tutorial-pt1

In [2]:
import ee

ee.Authenticate(auth_mode='notebook')
ee.Initialize()

In [3]:
# Import other packages used in the tutorial
%matplotlib inline
import geemap
import numpy as np
import random, time
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

from pprint import pprint  # for pretty printing

In [4]:
# Truncate a 1-D array to dec decimal places
def trunc(values, dec = 3):
    return np.trunc(values*10**dec)/(10**dec)

# Display an image in a one percent linear stretch
def display_ls(image, map, name, centered = False):
    bns = image.bandNames().length().getInfo()
    if bns == 3:
        image = image.rename('B1', 'B2', 'B3')
        pb_99 = ['B1_p99', 'B2_p99', 'B3_p99']
        pb_1 = ['B1_p1', 'B2_p1', 'B3_p1']
        img = ee.Image.rgb(image.select('B1'), image.select('B2'), image.select('B3'))
    else:
        image = image.rename('B1')
        pb_99 = ['B1_p99']
        pb_1 = ['B1_p1']
        img = image.select('B1')
    percentiles = image.reduceRegion(ee.Reducer.percentile([1, 99]), maxPixels=1e11)
    mx = percentiles.values(pb_99)
    if centered:
        mn = ee.Array(mx).multiply(-1).toList()
    else:
        mn = percentiles.values(pb_1)
    map.addLayer(img, {'min': mn, 'max': mx}, name)

In [6]:
aoi = ee.FeatureCollection(
    'projects/google/imad_tutorial/landkreis_olpe_aoi').geometry()

def collect(aoi, t1a ,t1b, t2a, t2b):
    try:
        im1 = ee.Image( ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
                               .filterBounds(aoi)
                               .filterDate(ee.Date(t1a), ee.Date(t1b))
                               .filter(ee.Filter.contains(rightValue=aoi,leftField='.geo'))
                               .sort('CLOUDY_PIXEL_PERCENTAGE')
                               .first()
                               .clip(aoi) )
        im2 = ee.Image( ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
                               .filterBounds(aoi)
                               .filterDate(ee.Date(t2a), ee.Date(t2b))
                               .filter(ee.Filter.contains(rightValue=aoi,leftField='.geo'))
                               .sort('CLOUDY_PIXEL_PERCENTAGE')
                               .first()
                               .clip(aoi) )
        timestamp = im1.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        timestamp = im2.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        return (im1, im2)
    except Exception as e:
        print('Error: %s'%e)

im1, im2 = collect(aoi, '2021-06-01', '2021-06-30', '2022-06-01', '2022-06-30')

Sun Jun 13 10:36:49 2021
Thu Jun 16 10:46:56 2022


In [8]:
# Interactive map
M1 = geemap.Map()
M1.centerObject(aoi, 11)

M1

Map(center=[51.08617675675536, 7.976478425048881], controls=(WidgetControl(options=['position', 'transparent_b…

In [9]:
visirbands = ['B2','B3','B4','B8','B11','B12']
visbands = ['B2','B3','B4']

diff = im1.subtract(im2).select(visbands)
display_ls(im1.select(visbands), M1, 'Image 1')
display_ls(im2.select(visbands), M1, 'Image 2')
display_ls(diff, M1, 'Difference')

In [10]:
def covarw(image, weights = None, scale = 20, maxPixels = 1e10):
    '''Return the weighted centered image and its weighted covariance matrix'''
    try:
        geometry = image.geometry()
        bandNames = image.bandNames()
        N = bandNames.length()
        if weights is None:
            weights = image.constant(1)
        weightsImage = image.multiply(ee.Image.constant(0)).add(weights)
        means = image.addBands(weightsImage) \
                    .reduceRegion(ee.Reducer.mean().repeat(N).splitWeights(),
                                scale = scale,
                                maxPixels = maxPixels) \
                    .toArray() \
                    .project([1])
        centered = image.toArray().subtract(means)
        B1 = centered.bandNames().get(0)
        b1 = weights.bandNames().get(0)
        nPixels = ee.Number(centered.reduceRegion(ee.Reducer.count(),
                                                scale = scale,
                                                maxPixels = maxPixels).get(B1))
        sumWeights = ee.Number(weights.reduceRegion(ee.Reducer.sum(),
                                                    geometry = geometry,
                                                    scale = scale,
                                                    maxPixels = maxPixels).get(b1))
        covw = centered.multiply(weights.sqrt()) \
                    .toArray() \
                    .reduceRegion(ee.Reducer.centeredCovariance(),
                                    geometry = geometry,
                                    scale = scale,
                                    maxPixels = maxPixels) \
                    .get('array')
        covw = ee.Array(covw).multiply(nPixels).divide(sumWeights)
        return (centered.arrayFlatten([bandNames]), covw)
    except Exception as e:
        print('Error: %s'%e)

In [11]:
def corr(cov):
    '''Transform covariance matrix to correlation matrix'''
    # diagonal matrix of inverse sigmas
    sInv = cov.matrixDiagonal().sqrt().matrixToDiag().matrixInverse()
    # transform
    corr = sInv.matrixMultiply(cov).matrixMultiply(sInv).getInfo()
    # truncate
    return [list(map(trunc, corr[i])) for i in range(len(corr))]

In [12]:
_, cov = covarw(im1.select(visirbands))
pprint(corr(cov))

[[1.0, 0.952, 0.949, 0.063, 0.647, 0.79],
 [0.952, 0.999, 0.927, 0.291, 0.772, 0.847],
 [0.949, 0.927, 1.0, 0.008, 0.74, 0.893],
 [0.063, 0.291, 0.008, 0.999, 0.486, 0.22],
 [0.647, 0.772, 0.74, 0.486, 0.999, 0.933],
 [0.79, 0.847, 0.893, 0.22, 0.933, 0.999]]


In [13]:
im12 = im1.select(visirbands).addBands(im2.select(visirbands))
_, covar = covarw(im12)
correl = np.array(corr(covar))
print(np.diag(correl[:6, 6:]))

[0.799 0.784 0.682 0.867 0.666 0.632]


In [14]:
def geneiv(C,B):
    '''Return the eigenvalues and eigenvectors of the generalized eigenproblem
                         C*X = lambda*B*X'''
    try:
        C = ee.Array(C)
        B = ee.Array(B)
        #  Li = choldc(B)^-1
        Li = ee.Array(B.matrixCholeskyDecomposition().get('L')).matrixInverse()
        #  solve symmetric, ordinary eigenproblem Li*C*Li^T*x = lambda*x
        Xa = Li.matrixMultiply(C) \
            .matrixMultiply(Li.matrixTranspose()) \
            .eigen()
        #  eigenvalues as a row vector
        lambdas = Xa.slice(1, 0, 1).matrixTranspose()
        #  eigenvectors as columns
        X = Xa.slice(1, 1).matrixTranspose()
        #  generalized eigenvectors as columns, Li^T*X
        eigenvecs = Li.matrixTranspose().matrixMultiply(X)
        return (lambdas, eigenvecs)
    except Exception as e:
        print('Error: %s'%e)

In [15]:
def mad_run(image1, image2, scale = 20):
    '''The MAD transformation of two multiband images'''
    try:
        image = image1.addBands(image2)
        region = image.geometry()
        nBands = image.bandNames().length().divide(2)
        centeredImage,covarArray = covarw(image,scale=scale)
        bNames = centeredImage.bandNames()
        bNames1 = bNames.slice(0,nBands)
        bNames2 = bNames.slice(nBands)
        centeredImage1 = centeredImage.select(bNames1)
        centeredImage2 = centeredImage.select(bNames2)
        s11 = covarArray.slice(0,0,nBands).slice(1,0,nBands)
        s22 = covarArray.slice(0,nBands).slice(1,nBands)
        s12 = covarArray.slice(0,0,nBands).slice(1,nBands)
        s21 = covarArray.slice(0,nBands).slice(1,0,nBands)
        c1 = s12.matrixMultiply(s22.matrixInverse()).matrixMultiply(s21)
        b1 = s11
        c2 = s21.matrixMultiply(s11.matrixInverse()).matrixMultiply(s12)
        b2 = s22
        # solution of generalized eigenproblems
        lambdas, A = geneiv(c1,b1)
        _,       B = geneiv(c2,b2)
        rhos = lambdas.sqrt().project(ee.List([1]))
        # MAD variances
        sigma2s = rhos.subtract(1).multiply(-2).toList()
        sigma2s = ee.Image.constant(sigma2s)
        # ensure sum of correlations between X and U is positive
        tmp = s11.matrixDiagonal().sqrt()
        ones = tmp.multiply(0).add(1)
        tmp = ones.divide(tmp).matrixToDiag()
        s = tmp.matrixMultiply(s11).matrixMultiply(A).reduce(ee.Reducer.sum(),[0]).transpose()
        A = A.matrixMultiply(s.divide(s.abs()).matrixToDiag())
        # ensure positive correlation between U and V
        tmp = A.transpose().matrixMultiply(s12).matrixMultiply(B).matrixDiagonal()
        tmp = tmp.divide(tmp.abs()).matrixToDiag()
        B = B.matrixMultiply(tmp)
        # canonical and MAD variates
        centeredImage1Array = centeredImage1.toArray().toArray(1)
        centeredImage2Array = centeredImage2.toArray().toArray(1)
        U = ee.Image(A.transpose()).matrixMultiply(centeredImage1Array) \
                    .arrayProject([0]) \
                    .arrayFlatten([bNames2])
        V = ee.Image(B.transpose()).matrixMultiply(centeredImage2Array) \
                    .arrayProject([0]) \
                    .arrayFlatten([bNames2])
        MAD = U.subtract(V)
        #  chi square image
        Z = MAD.pow(2) \
                .divide(sigma2s) \
                .reduce(ee.Reducer.sum()) \
                .clip(region)
        return (U, V, MAD, Z)
    except Exception as e:
        print('Error: %s'%e)

In [16]:
U, V, MAD, Z = mad_run(im1.select(visirbands), im2.select(visirbands))

In [17]:
M2 = geemap.Map()
M2.centerObject(aoi, 11)
display_ls(im1.select(visbands), M2, 'Image 1')
display_ls(im2.select(visbands), M2, 'Image 2')
display_ls(diff, M2, 'Difference')
display_ls(MAD.select(0, 1, 2), M2, 'MAD Image', True)

M2

Map(center=[51.08617675675536, 7.976478425048881], controls=(WidgetControl(options=['position', 'transparent_b…

In [18]:
_, covar = covarw(U.addBands(V))
correl = np.array(corr(covar))
pprint(correl)
print('rho =', np.diag(correl[:6,6:]))

array([[ 1.   ,  0.001, -0.   ,  0.   , -0.   ,  0.   ,  0.923,  0.   ,
        -0.   ,  0.   , -0.   , -0.   ],
       [ 0.001,  1.   ,  0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.864,
         0.   ,  0.   , -0.   , -0.   ],
       [-0.   ,  0.   ,  1.   ,  0.   ,  0.   , -0.   , -0.   ,  0.   ,
         0.707, -0.   ,  0.   , -0.   ],
       [ 0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        -0.   ,  0.642,  0.   ,  0.   ],
       [-0.   , -0.   ,  0.   ,  0.   ,  1.   ,  0.   , -0.   , -0.   ,
         0.   ,  0.   ,  0.536,  0.   ],
       [ 0.   , -0.   , -0.   ,  0.   ,  0.   ,  1.   , -0.   , -0.   ,
        -0.   ,  0.   ,  0.   ,  0.369],
       [ 0.923,  0.   , -0.   ,  0.   , -0.   , -0.   ,  0.999,  0.   ,
        -0.   ,  0.   , -0.   , -0.   ],
       [ 0.   ,  0.864,  0.   ,  0.   , -0.   , -0.   ,  0.   ,  1.   ,
         0.   ,  0.   , -0.   , -0.   ],
       [-0.   ,  0.   ,  0.707, -0.   ,  0.   , -0.   , -0.   ,  0.   ,
         1.   , -0.   , 

In [19]:
# display MAD covariance matrix
_, covar = covarw(MAD)
covar = covar.getInfo()
[list(map(trunc,covar[i])) for i in range(len(covar))]

[[0.078, 0.0, -0.0, 0.0, 0.0, 0.0],
 [0.0, 0.14, 0.0, -0.0, -0.0, 0.0],
 [-0.0, 0.0, 0.303, 0.0, -0.0, -0.0],
 [0.0, -0.0, 0.0, 0.371, 0.0, 0.0],
 [0.0, -0.0, -0.0, 0.0, 0.48, 0.0],
 [0.0, 0.0, -0.0, 0.0, 0.0, 0.654]]